In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [2]:
df = pd.read_csv('features.csv')

In [3]:
df.reset_index(drop = True,inplace = True)
df.drop(df.columns[0],axis=1 , inplace = True)
df

,product_id,seller_index,category_index,review_score
0,3880d25d502b15b1de6fddc42ad1d67a,1725,0,5.0
1,6dae4e6bc334685708571a48aa41d437,2809,0,3.0
2,41817d1d44000e3105f24470a5224bcb,2493,0,5.0
3,8173ca41cdb176462f9ae79821a48404,2870,0,5.0
4,6a0f59f3946c8169999e2fb7e946ac96,831,0,1.5
...,...,...,...,...
28281,cf6b03b7b6a0d499505b4342f06e5f80,776,70,3.0
28282,5f2ad53b751662e4c54dceffadb3804d,776,70,5.0
28283,f750e77aeeecafcb47516be7f8f278b8,65,70,5.0
28284,6c7a0a349ad11817745e3ad58abd5c79,2248,71,4.0


In [4]:
vector = df[['category_index','seller_index','review_score']].values
item_ids = df.index
product_id = df.product_id

In [5]:
def get_product(item_id):
    idx = item_ids[item_id]
    item_profile = vector[idx:idx+1]
    return item_profile
def get_related_products(ids):
    item_profile_list = [get_product(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profile_list)
    return item_profiles

In [6]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self,item_ids=None):
        self.item_ids = item_ids
    def get_model_name(self):
        return self.MODEL_NAME
    def _get_similar_products(self, item_id, topn=1000):
        cosine_similarities = cosine_similarity(get_product(item_id),vector)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_products = sorted([(item_ids[i],cosine_similarities[0,i]) for i in similar_indices ], key=lambda x : -x[1])
        return similar_products
    def get_recommendation(self,item_id,items_to_ignore = [], topn=10, verbose = False):
        items_to_ignore.append(item_id)
        similar_items = self._get_similar_products(item_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        return similar_items_filtered[:10]

In [7]:
csm = ContentBasedRecommender(item_ids)

In [36]:
csm = ContentBasedRecommender(item_ids)
x = csm.get_recommendation(26214)[:5]
ids = []
for i in range(0,len(x)):
    ids.append(x[i][0])
pids = []
for i in range (0,len(ids)):
    pids.append(product_id[ids[i]])
pids
product_id[13195]

'2f0b48ed3b5553d989aed89a6fcd9487'

In [28]:
df[df.product_id=="2f0b48ed3b5553d989aed89a6fcd9487"]

,product_id,seller_index,category_index,review_score
13195,2f0b48ed3b5553d989aed89a6fcd9487,232,7,1.0


In [26]:
df1 = pd.read_csv('olist_products_dataset.csv')


In [37]:
df1[df1.product_id=="2f0b48ed3b5553d989aed89a6fcd9487"]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
9855,2f0b48ed3b5553d989aed89a6fcd9487,automotivo,55.0,1698.0,1.0,6400.0,20.0,20.0,20.0
